# Выполним аналитику датасета.

При предпросмотре данных, было определено, что для некоторых отрывков "start" > "end", что не является логичным. Время конца отрывка не может быбть раньше времени его начала, поэтому сначала необходимо исследовать и исправить таргеты в ```test_labels.json``` и ```train_labels.json```. Всего нам дано 80 + 45 = 125 видео, определим видео с неверными временными метками заставки и исправим их на верные. Это будет будет ручная часть предподготовки данных, но она необходима, для корректного обучения модели.

In [11]:
#импорт базовых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import json
import os
from datetime import datetime

In [23]:
#алгоритм поиска неверных временных меток

#подгрузим данные
project_dir = '/kaggle/input/vk-series-data/'
test_target = os.path.join(project_dir, 'test_labels.json')
train_target = os.path.join(project_dir, 'train_labels.json')

with open(test_target) as f:
    test_target_dict = json.load(f)

with open(train_target) as f:
    train_target_dict = json.load(f)

def process_bad_times(dictionary):

    bad_videos = []

    for videos, description in dictionary.items():
        start_time = datetime.strptime(description["start"], "%H:%M:%S")
        end_time = datetime.strptime(description["end"], "%H:%M:%S")
        if start_time > end_time:
            bad_videos.append(description["url"])
            print("Ссылка:", description["url"], "время начала:",  description["start"], "время конца:" , description["end"])

    print(f"Количество неправильных временных меток заставок: {len(bad_videos)}")

print("Test")
process_bad_times(test_target_dict)
print("Train")
process_bad_times(train_target_dict)

Test
Ссылка: https://vkvideo.ru/video-220020068_456241671 время начала: 0:03:47 время конца: 0:03:10
Ссылка: https://vkvideo.ru/video-220020068_456256475 время начала: 0:10:42 время конца: 0:10:25
Ссылка: https://vkvideo.ru/video-220020068_456241758 время начала: 0:01:58 время конца: 0:01:02
Ссылка: https://vkvideo.ru/video-220020068_456255346 время начала: 0:01:51 время конца: 0:01:26
Ссылка: https://vkvideo.ru/video-220020068_456257137 время начала: 0:02:53 время конца: 0:02:00
Количество неправильных временных меток заставок: 5
Train
Ссылка: https://vkvideo.ru/video-220020068_456241671 время начала: 0:03:47 время конца: 0:03:10
Ссылка: https://vkvideo.ru/video-220020068_456255332 время начала: 0:02:57 время конца: 0:02:25
Ссылка: https://vkvideo.ru/video-220020068_456241672 время начала: 0:03:51 время конца: 0:03:15
Ссылка: https://vkvideo.ru/video-220020068_456256475 время начала: 0:10:42 время конца: 0:10:25
Ссылка: https://vkvideo.ru/video-220020068_456248657 время начала: 0:02:3

То есть получаем, что у 5+24 видео нелогичные временные метки. Эти метки необходимо исправить и так как автоматизированного подхода/написанной нейросети на данный момент не существует, то данную работу придется выполнять вручную. Ниже отметил ссылку на видео, неправильное время, правильное время.